In [14]:
# Imports
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from IPython.display import display
from time import time

In [15]:
def ftr_score(string):
    if string == 'H':
        return 0
    elif string == "A":
        return 1
    else:
        return 2

def last5_score(string):
    if string == 'N':
        return 0
    elif string == "W":
        return 1
    else:
        return 2

# Read data
data = pd.read_csv("C:/Users/theerik/PycharmProjects/predictor/data/final/final.csv")
display(data.head())

# change FTR letters to numbers
data['FTR'] = data.FTR.apply(ftr_score)
# change last 5 matches
data['HM1'] = data.HM1.apply(last5_score)
data['HM2'] = data.HM2.apply(last5_score)
data['HM3'] = data.HM3.apply(last5_score)
data['HM4'] = data.HM4.apply(last5_score)
data['HM5'] = data.HM5.apply(last5_score)

data['AM1'] = data.AM1.apply(last5_score)
data['AM2'] = data.AM2.apply(last5_score)
data['AM3'] = data.AM3.apply(last5_score)
data['AM4'] = data.AM4.apply(last5_score)
data['AM5'] = data.AM5.apply(last5_score)

nr = data.shape[0]
wins = len(data[data.FTR == 0])
away = len(data[data.FTR == 1])
draw = len(data[data.FTR == 2])
print("Home win rate {:.4f}%".format(float(wins / nr) * 100))
print("Away win rate {:.4f}%".format(float(away / nr) * 100))
print("Draw rate {:.4f}%".format(float(draw / nr) * 100))


,Unnamed: 0,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,...,HTLS5,ATWS3,ATWS5,ATLS3,ATLS5,HTGD,ATGD,DP,DFP,DLP
0,0,2020-09-12,Fulham,Arsenal,0,3,A,0,0,0,...,0,0,0,0,0,0,0,0,0,15
1,1,2020-09-12,Crystal Palace,Southampton,1,0,H,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2,2,2020-09-12,Liverpool,Leeds,4,3,H,0,0,0,...,0,0,0,0,0,0,0,0,0,-16
3,3,2020-09-12,West Ham,Newcastle,0,2,A,0,0,0,...,0,0,0,0,0,0,0,0,0,-9
4,4,2020-09-13,West Brom,Leicester,0,3,A,0,0,0,...,0,0,0,0,0,0,0,0,0,-2


Home win rate 45.1053%
Away win rate 32.3158%
Draw rate 22.5789%


In [16]:
# FTR = Full Time Result (H=Home Win: 0, A=Away Win: 1, D=Draw: 2)
# remove useless/cheat data
X_all = data.drop(['FTR', "FTHG", "FTAG", "Date", "Unnamed: 0", "HTFPS", "ATFPS"], axis=1)
y_all = data['FTR']

# # Standardising the data.
# from sklearn.preprocessing import scale
#
# # Center to the mean and component wise scale to unit variance.
# cols = [['HTGD','ATGD','HTP','ATP','DLP']]
# for col in cols:
#     X_all[col] = scale(X_all[col])
#
# display(X_all.head())

In [17]:
# display(X_all.head())

# we want continous vars that are integers for our input data, so lets remove any categorical vars
def preprocess_features(X):
    """
    Preprocesses football data and
    converts categorical variables into dummy variables.
    """
    # new dataframe
    output = pd.DataFrame(index = X.index)

    # iterate each column and find what type of data is in there
    for col, col_data in X.iteritems():

        # if data type is categorical (for example string),
        # convert to dummy/indicator variables (0's and 1's)
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)

        # add columns to output
        output = output.join(col_data)
    return output
X_all = preprocess_features(X_all)
print("Processed feature columns ({} total features):{}".format(len(X_all.columns), list(X_all.columns)))
print()
display(X_all.head())

Processed feature columns (94 total features):['HomeTeam_Arsenal', 'HomeTeam_Aston Villa', 'HomeTeam_Bournemouth', 'HomeTeam_Brighton', 'HomeTeam_Burnley', 'HomeTeam_Cardiff', 'HomeTeam_Chelsea', 'HomeTeam_Crystal Palace', 'HomeTeam_Everton', 'HomeTeam_Fulham', 'HomeTeam_Huddersfield', 'HomeTeam_Hull', 'HomeTeam_Leeds', 'HomeTeam_Leicester', 'HomeTeam_Liverpool', 'HomeTeam_Man City', 'HomeTeam_Man United', 'HomeTeam_Middlesbrough', 'HomeTeam_Newcastle', 'HomeTeam_Norwich', 'HomeTeam_Sheffield United', 'HomeTeam_Southampton', 'HomeTeam_Stoke', 'HomeTeam_Sunderland', 'HomeTeam_Swansea', 'HomeTeam_Tottenham', 'HomeTeam_Watford', 'HomeTeam_West Brom', 'HomeTeam_West Ham', 'HomeTeam_Wolves', 'AwayTeam_Arsenal', 'AwayTeam_Aston Villa', 'AwayTeam_Bournemouth', 'AwayTeam_Brighton', 'AwayTeam_Burnley', 'AwayTeam_Cardiff', 'AwayTeam_Chelsea', 'AwayTeam_Crystal Palace', 'AwayTeam_Everton', 'AwayTeam_Fulham', 'AwayTeam_Huddersfield', 'AwayTeam_Hull', 'AwayTeam_Leeds', 'AwayTeam_Leicester', 'AwayTe

,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Bournemouth,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Cardiff,HomeTeam_Chelsea,HomeTeam_Crystal Palace,HomeTeam_Everton,HomeTeam_Fulham,...,HTLS5,ATWS3,ATWS5,ATLS3,ATLS5,HTGD,ATGD,DP,DFP,DLP
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,15
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,6
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-16
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-9
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-2


In [18]:
from sklearn.model_selection import train_test_split

# make train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all,
    # test_size = 180,
    # train_size= 600,
    random_state = 1,
    shuffle=True,
    stratify = None
    # stratify = y_all
)

In [19]:
# F1 score (also F-score or F-measure) is a measure of a test's accuracy.
# It considers both the precision p and the recall r of the test to compute
# the score: p is the number of correct positive results divided by the number of
# all positive results, and r is the number of correct positive results divided by
# the number of positive results that should have been returned. The F1 score can be
# interpreted as a weighted average of the precision and recall, where an F1 score
# reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''

    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()

    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))


def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''

    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)

    end = time()
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))

    return f1_score(target, y_pred,  average='weighted'), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''

    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))

    # Train the classifier
    train_classifier(clf, X_train, y_train)

    # show results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))

    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))
    print()

seed = 0

clf_A = LogisticRegression(
    random_state=seed,
    max_iter=3000
)
clf_B = SVC(
    random_state=seed,
    kernel='rbf'
)
clf_C = xgb.XGBClassifier(
    seed=seed,
    eval_metric='mlogloss',
    use_label_encoder=False
)

# train_predict(clf_A, X_train, y_train, X_test, y_test)
train_predict(clf_B, X_train, y_train, X_test, y_test)
train_predict(clf_C, X_train, y_train, X_test, y_test)

Training a SVC using a training set size of 1425. . .
Trained model in 0.2440 seconds
Made predictions in 0.3460 seconds.
F1 score and accuracy score for training set: 0.4741 , 0.5537.
Made predictions in 0.1110 seconds.
F1 score and accuracy score for test set: 0.4628 , 0.5389.

Training a XGBClassifier using a training set size of 1425. . .
Trained model in 1.0900 seconds
Made predictions in 0.0130 seconds.
F1 score and accuracy score for training set: 1.0000 , 1.0000.
Made predictions in 0.0090 seconds.
F1 score and accuracy score for test set: 0.4937 , 0.5200.



In [20]:
# optimize xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


# # A parameter grid for XGBoost
# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [3, 4, 5]
#         }

parameters = {
    'min_child_weight': [3],
    'gamma':[0.4],
    'subsample' : [0.8],
    'colsample_bytree' : [0.8],
    'max_depth': [4],  # 3
    'learning_rate' : [0.1],
    'n_estimators' : [40],
    # 'scale_pos_weight' : [1],
    'reg_alpha':[1e-5],
    # "average": ['weighted']
}

clf = xgb.XGBClassifier(
    seed=seed,
    booster="gbtree",
    validate_parameters=False,
    # average= 'weighted',
    # metric='multiclass',
    eval_metric='mlogloss',
    num_class=3,
    objective="multi:softmax",
    use_label_encoder=False
)

f1_scorer = make_scorer(
    f1_score,
    average='weighted'
)

grid_obj = GridSearchCV(
    clf,
    scoring=f1_scorer,
    param_grid=parameters,
    cv=5
)

# # TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)

# Report the final F1 score for training and testing after parameter tuning
f1, acc = predict_labels(clf, X_train, y_train)
print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))

f1, acc = predict_labels(clf, X_test, y_test)
print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eval_metric='mlogloss',
              gamma=0.4, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=4, min_child_weight=3, missing=nan,
              monotone_constraints='()', n_estimators=40, n_jobs=4, num_class=3,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=None, seed=0,
              subsample=0.8, tree_method='exact', use_label_encoder=False,
              validate_parameters=False, ...)
Made predictions in 0.0140 seconds.
F1 score and accuracy score for training set: 0.6082 , 0.6498.
Made predictions in 0.0120 seconds.
F1 score and accuracy score for test set: 0.4993 , 0.5579.


In [21]:
clf.save_model("model.txt")

print(clf)

# model_xgb_2 = xgb.XGBClassifier()
# model_xgb_2.load_model("model.txt")
#
#
# print(model_xgb_2)
#
# # Report the final F1 score for training and testing after parameter tuning
# f1, acc = predict_labels(model_xgb_2, X_train, y_train)
# print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
#
# f1, acc = predict_labels(model_xgb_2, X_test, y_test)
# print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eval_metric='mlogloss',
              gamma=0.4, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=4, min_child_weight=3, missing=nan,
              monotone_constraints='()', n_estimators=40, n_jobs=4, num_class=3,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=None, seed=0,
              subsample=0.8, tree_method='exact', use_label_encoder=False,
              validate_parameters=False, ...)
